In [1]:
import pandas as pd
from openfisca_ceq.tools.survey_scenario import build_ceq_survey_scenario
from openfisca_ceq.tools.indirect_taxation.tax_benefit_system_indirect_taxation_completion import indirect_tax_by_country
from openfisca_ceq.tools.data import year_by_country
from openfisca_ceq.tools.data_ceq_correspondence import (
    ceq_input_by_harmonized_variable,
    ceq_intermediate_by_harmonized_variable,
    non_ceq_input_by_harmonized_variable,
    )

# Build country scenario

## Country selection

In [2]:
country = "mali"
year = year_by_country[country]

In [3]:
survey_scenario = build_ceq_survey_scenario(legislation_country = country, year = year)

# Aggegates check

## Income variables

### Disagregated income variables

In [4]:
ceq_by_harmonized_variable = dict()
ceq_by_harmonized_variable.update(ceq_input_by_harmonized_variable)
ceq_by_harmonized_variable.update(ceq_intermediate_by_harmonized_variable)
ceq_by_harmonized_variable.update(non_ceq_input_by_harmonized_variable)
ceq_by_harmonized_variable

{'rev_i_autoconsommation': 'autoconsumption',
 'rev_i_autres': 'other_income',
 'rev_i_autres_transferts': 'gifts_sales_durables',
 'rev_i_loyers_imputes': 'imputed_rent',
 'rev_i_transferts_publics': 'direct_transfers',
 'rev_i_agricoles': 'revenu_agricole',
 'rev_i_autres_revenus_capital': 'autres_revenus_du_capital',
 'rev_i_independants_Ntaxe': 'revenu_informel_non_salarie',
 'rev_i_independants_taxe': 'revenu_non_salarie',
 'rev_i_independants': 'revenu_non_salarie_total',
 'rev_i_locatifs': 'revenu_locatif',
 'rev_i_pensions': 'pension_retraite',
 'rev_i_salaires_formels': 'salaire',
 'rev_i_salaires_informels': 'revenu_informel_salarie'}

In [5]:
data = [
    (harmonized_variable, openfisca_variable, survey_scenario.compute_aggregate(openfisca_variable, period = year) / 1e9) 
    for harmonized_variable, openfisca_variable in ceq_by_harmonized_variable.items()
    ]

In [6]:
revenus = pd.DataFrame(data, columns = ["harmonized", "openfisca", "aggregate"])
revenus

,harmonized,openfisca,aggregate
0,rev_i_autoconsommation,autoconsumption,279.103047
1,rev_i_autres,other_income,270.665025
2,rev_i_autres_transferts,gifts_sales_durables,77.723609
3,rev_i_loyers_imputes,imputed_rent,153.277072
4,rev_i_transferts_publics,direct_transfers,0.000000
5,rev_i_agricoles,revenu_agricole,621.465183
6,rev_i_autres_revenus_capital,autres_revenus_du_capital,0.000000
7,rev_i_independants_Ntaxe,revenu_informel_non_salarie,260.731579
8,rev_i_independants_taxe,revenu_non_salarie,260.731579
9,rev_i_independants,revenu_non_salarie_total,521.463159


### Agregated income variables

#### CEQ agregated income variables

In [7]:
ceq_variables = ["market_income", "market_income_plus_pensions"]

In [8]:
ceq_aggregates = pd.DataFrame(
    index = ceq_variables,
    columns = ['aggregate'],
    data = [
        survey_scenario.compute_aggregate(ceq_variable, period = year) / 1e9 
        for ceq_variable in ceq_variables
        ]
    )
ceq_aggregates

,aggregate
market_income,2190.30853
market_income_plus_pensions,2190.30853


#### Direct check

In [9]:
revenu_components_summed_in_total = list(ceq_by_harmonized_variable.values()) 
revenu_components_summed_in_total.remove("revenu_non_salarie_total")
revenu_components_summed_in_total.remove("direct_transfers")
total_revenu = sum(survey_scenario.compute_aggregate(revenu_component, period = survey_scenario.year) for revenu_component in revenu_components_summed_in_total) / 1e9
total_revenu

2190.3085312195262

In [10]:
from numpy.testing import assert_almost_equal
assert_almost_equal(ceq_aggregates.loc['market_income_plus_pensions'].values, total_revenu, decimal = 5) 

## Education variables

### Counts (in millions)

In [11]:
print((survey_scenario.compute_pivot_table(columns = ['eleve_enseignement_niveau'], index = ['eleve_enseignement_public'], aggfunc = 'count', period = survey_scenario.year) / 1e6).round(2))

eleve_enseignement_niveau  -1.0   0.0   1.0   2.0   3.0
eleve_enseignement_public                              
1.0                        12.7  0.11  1.82   1.3  0.12


In [12]:
variables = [
    'pre_school_person',
    'pre_school',
    'primary_education_person',
    'primary_education',
    'secondary_education_person',
    'secondary_education',
    'tertiary_education_person',
    'tertiary_education',
    'education_net_transfers',
    ]
print("In kind education tranfers")
for variable in variables:
    print(
        "{variable}: {aggregate} billions FCFA".format(
            variable = variable,
            aggregate = int(round(survey_scenario.compute_aggregate(variable, period = survey_scenario.year) / 1e9))
            )
        )

In kind education tranfers
pre_school_person: 0 billions FCFA
pre_school: 0 billions FCFA
primary_education_person: 123 billions FCFA
primary_education: 123 billions FCFA
secondary_education_person: 183 billions FCFA
secondary_education: 183 billions FCFA
tertiary_education_person: 16 billions FCFA
tertiary_education: 16 billions FCFA
education_net_transfers: 322 billions FCFA


## Tax variables

### Indirect tax variables

In [13]:
indirect_tax_variables = [
    variable 
    for tax in indirect_tax_by_country[country]
    for variable in survey_scenario.tax_benefit_system.variables.keys() 
    if tax in variable 
    ]
indirect_tax_variables

['depenses_ht_tva_exonere',
 'depenses_ht_tva_taux_normal',
 'depenses_ht_tva_taux_reduit',
 'tva_taux_normal',
 'tva_taux_reduit',
 'tva']

In [14]:
 pd.DataFrame(
    index = indirect_tax_variables,
    columns = ['aggregate'],
    data = [
        survey_scenario.compute_aggregate(variable, period = year) / 1e9 
        for variable in indirect_tax_variables
        ]
    )

,aggregate
depenses_ht_tva_exonere,1227.968785
depenses_ht_tva_taux_normal,1266.411291
depenses_ht_tva_taux_reduit,0.404737
tva_taux_normal,227.954042
tva_taux_reduit,0.020237
tva,227.974278


In [15]:
survey_scenario.tax_benefit_system.variables.keys()

dict_keys(['conjoint_a_des_revenus', 'salaire', 'household_weight', 'person_weight', 'age', 'date_naissance', 'marie', 'impot_brut', 'impot_traitement_salaire', 'nombre_enfants_a_charge', 'reduction_charge_famille', 'revenu_net_imposable', 'alimony', 'autoconsumption', 'consumable_income', 'disposable_income', 'final_income', 'gifts_sales_durables', 'gross_income', 'imputed_rent', 'market_income', 'market_income_plus_pensions', 'net_market_income', 'other_income', 'taxable_income', 'agricultural_inputs_subsidies', 'electricity_subsidies', 'food_subsidies', 'fuel_subsidies', 'indirect_subsidies', 'contributions_health', 'contributions_pensions', 'employee_contributions_health', 'employee_contributions_pensions', 'employee_other_contributions', 'employer_contributions_health', 'employer_contributions_pensions', 'employer_other_contributions', 'other_contributions', 'self_employed_contributions_health', 'self_employed_contributions_pensions', 'self_employed_other_contributions', 'corporat